In [ ]:
import heapq
import random


## MARBLE SOLITAIRE BOARD
import numpy as np
# mat = np.ones((7, 7))*-1
mat = [[2]*7]*7
for i in range(3):
  for j in range(3):
    for k in range(3):
      for l in range(3):
        if(i==1 or j==1):
          mat[i*2+k][j*2+l] = 1
mat[3][3] = 0


from typing import List, Tuple
class Board(object):
    def __init__(self, board=None):
      # 0 = empty, 1 = occupied, 2 = invalid
      self.board = board or [
        [2, 2, 1, 1, 1, 2, 2],
        [2, 2, 1, 1, 1, 2, 2],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [2, 2, 1, 1, 1, 2, 2],
        [2, 2, 1, 1, 1, 2, 2],
      ]
      # self.board = board

    def __hash__(self):
      # return hash(tuple([tuple(row) for row in self.board]))
      return hash(tuple(tuple(row) for row in self.board))

    def clone(self):
      # Ca. 2x faster than copy.deepcopy()
      board_copy = [[peg for peg in row] for row in self.board]
      return Board(board_copy)

    def score(self) -> int:
      # Count all pegs
      s = 0
      for peg in [p for row in self.board for p in row]:
          s += 1 if peg == 1 else 0

      # If there is only one peg left and it is in the center, the score is 0.
      if s == 1 and self.board[3][3] == 1:
          s = 0

      return s

    def possible_moves(self) -> List[Tuple[Tuple[int, int], Tuple[int, int]]]:
      moves = []  # Format: ((from x, from y), (to x, to y))

      # For each board position
      for x, row in enumerate(self.board):
          for y, peg in enumerate(row):
              # If occupied by a peg
              if peg == 1:
                  # Find valid moves for this peg
                  peg_moves = self.moves_for_peg(x, y)

                  moves.extend([((x, y), move) for move in peg_moves])

      return moves

    def moves_for_peg(self, x, y) -> List[Tuple[int, int]]:
      assert 0 <= x <= 6
      assert 0 <= y <= 6

      peg_moves = []

      # x, y offsets for moves towards top, bottom, left, and right
      move_offsets = [(-2, 0), (2, 0), (0, -2), (0, 2)]

      for (dx, dy) in move_offsets:
        new_x = x + dx
        new_y = y + dy

        # If new position is inside the board
        if 0 <= new_x <= 6 and 0 <= new_y <= 6:
          # If the new position is free
          if self.board[new_x][new_y] == 0:
            # If there is a peg next to the current peg in the move's direction
            if self.board[(x + new_x) // 2][(y + new_y) // 2] == 1:
              peg_moves.append((new_x, new_y))

      return peg_moves

    def move(self, move: Tuple[Tuple[int, int], Tuple[int, int]]):
      (from_x, from_y), (to_x, to_y) = move

      # Delete peg from old position
      assert self.board[from_x][from_y] == 1
      self.board[from_x][from_y] = 0
      # Place peg at new position
      assert self.board[to_x][to_y] == 0
      self.board[to_x][to_y] = 1
      # Delete peg in between
      assert self.board[(from_x + to_x) // 2][(from_y + to_y) // 2] == 1
      self.board[(from_x + to_x) // 2][(from_y + to_y) // 2] = 0

      return self

# Set of hashes of board positions. Used to skip boards that have been played already.
boards_played = set()

# Counters for statistical purposes.
statistics = {'Games finished': 0, 'Boards skipped': 0}


def solve_recursive(board, move_memo=()):
  if hash(board) in boards_played:
    statistics['Boards skipped'] += 1
    return
  boards_played.add(hash(board))

  moves = board.possible_moves()

    # If there are no moves left
  if len(moves) == 0:
    statistics['Games finished'] += 1

    # If the game is solved
    if board.score() == 0:
      return move_memo
  else:
    for move in moves:
      result = solve_recursive(board.clone().move(move), [mm for mm in move_memo] + [move])
      if result:
        return result

if __name__ == '__main__':
  moves_played = solve_recursive(Board())
  print(f"Finished {statistics['Games finished']} games! (skipped {statistics['Boards skipped']})")
  if moves_played:
    m = '\n'.join([f"{m[0][0]}, {m[0][1]} -> {m[1][0]}, {m[1][1]}" for m in moves_played])
    print(f"Solution found, moves:\n{m}")

import heapq

# Define the initial board configuration (0: empty, 1: marble)
initial_board = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
]

# Define goal state (1: marble at the center)
goal_board = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

# Define possible moves (up, down, left, right)
moves = [(0, -2), (0, 2), (-2, 0), (2, 0)]

# Define the heuristic functions
def heuristic_1(board):
    return sum(row.count(1) for row in board)

def heuristic_2(board):
    center_x, center_y = 2, 2
    return sum(abs(i - center_x) + abs(j - center_y) for i, row in enumerate(board) for j, cell in enumerate(row) if cell == 1)

# Define the Node class for representing states in the search
class Node:
    def __init__(self, board, cost, heuristic):
        self.board = board
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.heuristic) < (other.heuristic)

# Implement Best-First Search Algorithm
def best_first_search(initial_board, heuristic_func):
    heap = [Node(initial_board, 0, heuristic_func(initial_board))]
    heapq.heapify(heap)
    visited = set()

    while heap:
        current_node = heapq.heappop(heap)
        current_board = current_node.board

        if current_board == goal_board:
            return current_node.cost

        if tuple(map(tuple, current_board)) in visited:
            continue

        visited.add(tuple(map(tuple, current_board)))

        for move_x, move_y in moves:
            new_board = [row[:] for row in current_board]
            for i in range(len(new_board)):
                for j in range(len(new_board[0])):
                    if 0 <= i + move_x < 5 and 0 <= j + move_y < 5 and \
                       0 <= i + move_x * 2 < 5 and 0 <= j + move_y * 2 < 5 and \
                       new_board[i][j] == 1 and new_board[i + move_x][j + move_y] == 1 and new_board[i + move_x * 2][j + move_y * 2] == 0:
                        new_board[i][j] = 0
                        new_board[i + move_x][j + move_y] = 0
                        new_board[i + move_x * 2][j + move_y * 2] = 1
                        heapq.heappush(heap, Node(new_board, current_node.cost + 1, heuristic_func(new_board)))

# Execute Best-First Search with Heuristic 1
result_bfs_heuristic_1 = best_first_search(initial_board, heuristic_1)
print(f"Best-First Search with Heuristic 1: {result_bfs_heuristic_1}")


# Implement A* Algorithm
def a_star_search(initial_board, heuristic_func):
    heap = [Node(initial_board, 0, heuristic_func(initial_board))]
    heapq.heapify(heap)
    visited = set()

    while heap:
        current_node = heapq.heappop(heap)
        current_board = current_node.board

        if current_board == goal_board:
            return current_node.cost

        if tuple(map(tuple, current_board)) in visited:
            continue

        visited.add(tuple(map(tuple, current_board)))

        for move_x, move_y in moves:
            new_board = [row[:] for row in current_board]
            for i in range(len(new_board)):
                for j in range(len(new_board[0])):
                    if 0 <= i + move_x < 5 and 0 <= j + move_y < 5 and \
                       0 <= i + move_x * 2 < 5 and 0 <= j + move_y * 2 < 5 and \
                       new_board[i][j] == 1 and new_board[i + move_x][j + move_y] == 1 and new_board[i + move_x * 2][j + move_y * 2] == 0:
                        new_board[i][j] = 0
                        new_board[i + move_x][j + move_y] = 0
                        new_board[i + move_x * 2][j + move_y * 2] = 1
                        heapq.heappush(heap, Node(new_board, current_node.cost + 1, heuristic_func(new_board)))

# Execute A* Search with Heuristic 1
result_astar_heuristic_1 = a_star_search(initial_board, heuristic_1)
print(f"A* Search with Heuristic 1: {result_astar_heuristic_1}")

# Execute Best-First Search with Heuristic 1
result_bfs_heuristic_1 = best_first_search(initial_board, heuristic_1)
print(f"Best-First Search with Heuristic 1: {result_bfs_heuristic_1}")

# Execute Best-First Search with Heuristic 2
result_bfs_heuristic_2 = best_first_search(initial_board, heuristic_2)
print(f"Best-First Search with Heuristic 2: {result_bfs_heuristic_2}")

# Execute A* Search with Heuristic 1
result_astar_heuristic_1 = a_star_search(initial_board, heuristic_1)
print(f"A* Search with Heuristic 1: {result_astar_heuristic_1}")

# Execute A* Search with Heuristic 2
result_astar_heuristic_2 = a_star_search(initial_board, heuristic_2)
print(f"A* Search with Heuristic 2: {result_astar_heuristic_2}")

import sys
import time

class Board:
    def __init__(self, board_size):
        self.board_size = board_size
        self.board = [['-' for _ in range(board_size)] for _ in range(board_size)]
        self.peg_count = board_size**2
        self.goal_state = [['-' for _ in range(board_size)] for _ in range(board_size)]
        self.goal_state[board_size//2][board_size//2] = 'G'
        self.goal_state_found = False
        self.generate_board()

    def generate_board(self):
        for row in range(self.board_size):
            for col in range(self.board_size):
                if row == 0 or row == self.board_size-1 or col == 0 or col == self.board_size-1:
                    self.board[row][col] = 'W'
                else:
                    self.board[row][col] = str(self.peg_count)
                    self.peg_count -= 1

    def print_board(self):
        for row in self.board:
            print(' '.join(row))

    def is_goal(self):
        if self.goal_state_found:
            return True
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.board[row][col] != self.goal_state[row][col]:
                    return False
        self.goal_state_found = True
        return True

    def successors(self):
        successors = []
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.board[row][col] != '-':
                    for move in self.get_valid_moves(row, col):
                        new_board = self.board.copy()
                        new_board[row][col] = '-'
                        new_board[move[0]][move[1]] = self.board[row][col]
                        successors.append((move, Board(new_board)))
        return successors

    def get_valid_moves(self, row, col):
        moves = []
        if row > 0 and self.board[row-1][col] == '-':
            moves.append((row-1, col))
        if row < self.board_size-1 and self.board[row+1][col] == '-':
            moves.append((row+1, col))
        if col > 0 and self.board[row][col-1] == '-':
            moves.append((row, col-1))
        if col < self.board_size-1 and self.board[row][col+1] == '-':
            moves.append((row, col+1))
        if row > 0 and col > 0 and self.board[row-1][col-1] == '-':
            moves.append((row-1, col-1))
        if row > 0 and col < self.board_size-1 and self.board[row-1][col+1] == '-':
            moves.append((row-1, col+1))
        if row < self.board_size-1 and col > 0 and self.board[row+1][col-1] == '-':
            moves.append((row+1, col-1))
        if row < self.board_size-1 and col < self.board_size-1 and self.board[row+1][col+1] == '-':
            moves.append((row+1, col+1))
        return moves

def best_first_search(board):
    frontier = [(0, board)]
    explored = []
    while frontier:
        fscore, current_board = frontier.pop(0)
        if current_board.is_goal():
            return current_board
        for move, child_board in current_board.successors():
            if child_board not in explored:
                gscore = fscore + 1
                frontier.append((gscore, child_board))
        explored.append(current_board)

if __name__ == '__main__':
    board_size = int(input("Enter the size of the board: "))
    board = Board(board_size)
    start = time.time()
    solution = best_first_search(board)
    end = time.time()
    if solution:
        print("Solution found!")
        solution.print_board()
        print("Time taken:", end-start, "seconds")
    else:
        print("No solution found!")

import random

def generate_random_k_sat(k, m, n):
    if k > n:
        print("Error: k must be less than or equal to n.")
        return

    clauses = []
    variables = [i + 1 for i in range(n)]

    for _ in range(m):
        clause = []
        while len(clause) < k:
            variable = random.choice(variables)
            negate = random.choice([True, False])
            if negate:
                variable = -variable
            if variable not in clause and -variable not in clause:
                clause.append(variable)

        clauses.append(clause)

    return clauses

def print_k_sat_problem(clauses):
    for clause in clauses:
        print(" ".join(map(str, clause)))

# Example usage
k_value = 3  # Change this to the desired k value
m_value = 5  # Change this to the desired number of clauses (m)
n_value = 4  # Change this to the desired number of variables (n)

random_k_sat_instance = generate_random_k_sat(k_value, m_value, n_value)
print_k_sat_problem(random_k_sat_instance)

def generate_k_sat(k, m, n):
  clauses = []
  for _ in range(m):
    clause = []

    while len(clause) < k:
      variable = random.randint(1, n)
      negated = random.choice([True, False])
      if negated:
        variable = -variable
      if variable not in clause:
        clause.append(variable)
    clause_str = " v ".join(map(str, clause))
    clauses.append(f"({clause_str})")
  return " ∧ ".join(clauses)


k = int(input("Enter the value of k- "))
m = int(input("Enter number of clauses- "))
n = int(input("Enter the number of variables- "))
clauses = generate_k_sat(k, m, n)


print("Generated k-SAT problem:")
print(clauses)

from string import ascii_lowercase
import random
from itertools import combinations
import numpy as np

# %%
print("Enter the number of variables in a clause: k = ")
k = int(input())
print("Enter the number of clauses : m =")
m = int(input())
print("Enter number of variables: n = ")
n = int(input())

# %%
def createProblem(m, k, n):
    #lower Case +ve
    positive_var = (list(ascii_lowercase))[:n]
    negative_var = [c.upper() for c in positive_var]
    variables = positive_var + negative_var
    total_problems = 10
    problems = []
    allCombs = list(combinations(variables, k))
    i = 0

    while i<total_problems:
        c = random.sample(allCombs, m)
        if c not in problems:
            i += 1
            problems.append(list(c))
    return variables, problems

# %%
variables, problems = createProblem(m, k, n)

# %%
def assignment(variables, n):
    forPositive = list(np.random.choice(2,n))
    forNegative = [abs(1-i) for i in forPositive]
    assign = forPositive + forNegative
    var_assign = dict(zip(variables, assign))
    return var_assign

# %%
assign = assignment(variables, n)
print(assign)
print(problems[0])

# %%
def solve(problem, assign):
    count = 0
    for sub in problem:
        l = [assign[val] for val in sub]
        count += any(l)
    return count

# %%
def hillClimbing(problem, assign, parentNum, received, step):

    bestAssign = assign.copy()
    assignValues = list(assign.values())
    assignKeys = list(assign.keys())

    maxNum = parentNum
    maxAssign = assign.copy()
    editAssign = assign.copy()

    for i in range(len(assignValues)):
        step += 1
        editAssign[assignKeys[i]] = abs(assignValues[i]-1)
        c = solve(problem, editAssign)
        if maxNum<c:
            received = step
            maxNum = c
            maxAssign = editAssign.copy()
    if maxNum==parentNum:
        s = str(received) + "/" + str(step)
        return bestAssign, maxNum, s
    else:
        parentNum = maxNum
        bestassign = maxAssign.copy()
        return hillClimbing(problem, bestassign, parentNum, received, step)

# %%
def beamSearch(problem, assign,  b, stepSize):

    bestAssign = assign.copy()
    assignValues = list(assign.values())
    assignKeys = list(assign.keys())
    steps = []
    possibleAssigns = []
    possibleScores = []

    editAssign = assign.copy()

    initail = solve(problem, assign)
    if initial == len(problem):
        p = str(stepSize) + "/" + str(stepSize)
        return assign, p

    for i in range(len(assignValues)):
        stepSize += 1
        editAssign[assignKeys[i]] = abs(assignValues[i]-1)
        c = solve(problem, editAssign)
        possibleAssigns.append(editAssign.copy())
        possibleScores.append(c)
        steps.append(stepSize)

    selected = list(np.argsort(possibleScores))[-b:]

    if len(problem) in possibleScores:
        index = [i for i in range(len(possibleScores)) if possibleScores[i]==len(problem)]
        p = str(steps[index[0]]) + "/" + str(steps[-1])
        return possibleAssigns[ index[0] ], p
    else:
        selectedAssigns = [possibleAssigns[i] for i in selected]
        for a in selectedAssigns:
            return beamSearch(problem, a, b, stepSize)

# %%
def variableNeighbor(problem, assign, b, step):
    bestAssign = assign.copy()
    assignValues = list(assign.values())
    assignKeys = list(assign.keys())
    steps = []
    possibleAssigns = []
    possibleScores = []

    editAssign = assign.copy()

    initail = solve(problem, assign)
    if initial == len(problem):
        p = str(step) + "/" + str(step)
        return assign, p, b

    for i in range(len(assignValues)):
        step += 1
        editAssign[assignKeys[i]] = abs(assignValues[i]-1)
        c = solve(problem, editAssign)
        possibleAssigns.append(editAssign.copy())
        possibleScores.append(c)
        steps.append(step)

    selected = list(np.argsort(possibleScores))[-b:]

    if len(problem) in possibleScores:
        index = [i for i in range(len(possibleScores)) if possibleScores[i]==len(problem)]
        p = str(steps[index[0]]) + "/" + str(step)
        return possibleAssigns[index[0]], p, b

    else:
        selectedAssigns = [possibleAssigns[i] for i in selected]
        for a in selectedAssigns:
            return variableNeighbor(problem, a, b+1, step)

# %%
hAssigns = []
assigns = []
h_n = []
initials = []
hill_penetration = []
beam_penetration = []
var_penetration = []
v_n = []
b_var = []
b_n = []
bAssigns = []
vAssigns = []
i = 0

for problem in problems:
    i += 1
    l =[]
    assign = assignment(variables, n)
    initial = solve(problem, assign)
    bestAssign, score, hp = hillClimbing(problem, assign, initial, 1, 1)
    hAssigns.append(bestAssign)
    assigns.append(assign)
    h_n.append(score)
    initials.append(initial)
    hill_penetration.append(hp)
    # beam size = 3
    h, b3p = beamSearch(problem, assign, 3, 1)
    bAssigns.append(h)
    beam_penetration.append(b3p)
    # beam size = 3
    h4, b4p = beamSearch(problem, assign, 4, 1)

    v, p, bb = variableNeighbor(problem, assign, 3, 13)
    var_penetration.append(p)
    b_var.append(bb)
    vAssigns.append(v)

    print('Problem ',i,': ',problem)
    print('HillClimbing: ',bestAssign,', Penetrance:', hp)
    print('Beam search (width = 3): ',h,', Penetrance:', b3p)
    print('Beam search (width = 4): ', h4,', Penetrance:',b4p)
    print('Variable Neighbourhood: ',v,', Penetrance:',p)
    print()

# %%